In [ ]:
import pandas as pd
import re

In [ ]:
# Настройки отображения датафреймов
pd.set_option('display.max_rows', None)  # Показывать все строки
pd.set_option('display.max_columns', None)  # Показывать все столбцы
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Показывать полное содержимое ячеек
pd.set_option('display.expand_frame_repr', True)  # Разрешить перенос строк

In [ ]:
class MarketImpactClassifier:
    def __init__(self):
        # Добавляем список слов для исключения
        self.exclusion_keywords = [
            # 1. Криптовалюты
            r'\bкриптовалют', r'\bкрипта\b', r'\bбиткоин',
            r'\bbitcoin', r'\bэфир(иум)?', r'\beth(ereum)?',
            r'\bnft\b', r'\bдеф[аи]\b', r'\bстейблкоин',
            r'\bstablecoin', r'\bмайнинг', r'\b(mining)\b',
            r'\bбинанс\b', r'\bbinance\b', r'\bх[уа]оби\b',
            r'\bbybit\b', r'\bokx\b',

            # 2. Спорт, соревнования
            r'\bчемпионат', r'\bтурнир', r'\bматч[аи]?', r'\bлига\b', 
            r'\bфутбол', r'\bбаскетбол', r'\bхоккей', r'\bволейбол',
            r'\bформул[аи]-?\d', r'\bmotogp\b', r'\bufc\b',
            r'\bбокс\b', r'\bпромоутер\b',

            # 3. Развлечения, шоу-бизнес
            r'\bконцерт', r'\bфестивал', r'\bсериал', r'\bкино\b', 
            r'\bфильм', r'\bакт[её]р', r'\bактрис', r'\bпев[иец|ица]', 
            r'\bартист', r'\bшоубизнес', r'\bcelebrity', r'\bселебрити', 
            r'\bзвезд[аыя]', r'\bскандал', r'\bмузык[а-я]*\b',
            r'\bевровидени[еяю]\b',

            # 4. Гемблинг, казино
            r'\bказино\b', r'\bбукмекер', r'\bставк[иауе]\s+на\s+спорт',
            r'\bлотере[яию]', r'\bрулетк[а-я]*', r'\bигров[а-я]*\s+автомат',

            # 5. Авто/мотоспорт
            r'\bmotorsport', r'\bралли\s+рейд', r'\bавтоспорт',
            r'\bавтогонки', r'\bmotocross', r'\bsuperbike',
            r'\bmotogp\b', r'\bdrift\b', r'\bдрифт',

            # 6. Светская хроника
            r'\bсплетни\b', r'\bслухи\b', r'\bтаблоид\b',
            r'\bжелтая\s+пресса', r'\bgossip\b',

            # 7. Зарубежные автопроизводители
            r'\btesla\b', r'\bмерседес\b', r'\bmercedes\b', r'\bbmw\b',
            r'\baudi\b', r'\btoyota\b', r'\bnissan\b', r'\bhonda\b',

            # 8. Другие нерелевантные темы
            r'\bмода\b', r'\bfashion\b', r'\bкосметик[а-я]*\b',
            r'\bпокер\b', r'\bbst\b', r'\bастролог', r'\bгороскоп'

            # 9. Война
            r'НАТО', r'ATACMS', r'удары', r'бомб[аы]', r'ракет[аы]',
            r'войск', r'НАТО.',
        ]

        # Голубые фишки и их вариации названий
        self.blue_chips = [
            # Нефть и газ
            r'газпром', r'газпрома?', 
            r'(пао\s+)?газпром\s+нефть',
            r'лукойл', r'лукойла?', 
            r'роснефть', r'роснефти', 
            r'новатэк', r'новатэка?', 
            r'транснефть', r'транснефти',
            r'сургутнефтегаз',  # можно оставить так, чтобы точно ловить
            r'татнефть', r'татнефти',
            
            # Банки и финансы
            r'сбербанк', r'сбера?', 
            r'втб', 
            r'тинькофф', r'тинькова?',
            r'московск[ауюия]{2,3}\s+бирж[ауия]', r'мосбирж[ауия]', r'моех',

            # Металлургия и добыча
            r'норникель',
            # если реально в статьях «ГМК» = «Норникель», то используйте \b
            # r'\bгмк\b',  
            r'северсталь', r'северстали',
            r'полюс', r'полюса?',
            r'полиметалл', r'полиметалла?',
            r'алроса', r'алросы',
            r'нлмк', 
            r'магнит', r'магнита?',
            r'русал', r'русала?',
            r'фосагро',

            # Телекоммуникации
            r'мтс', 
            r'ростелеком', r'ростелекома?',

            # Другие крупные компании (IT, ритейл, авиалинии и пр.)
            r'яндекс', r'яндекса?',
            r'озон', r'озона?',
            r'аэрофлот', r'аэрофлота?',
            r'mail\.ru', r'vk',

            # Дополнительно при желании:
            # r'x5', r'х5', r'х5\s+retail',  # X5 Retail Group
            # r'лента', r'ленты?',
            # r'(м\.?видео|m\.?video)',  # М.Видео
            # r'детский\s+мир', r'детского\s+мира',
            # r'русагро', r'русагро?',
            # r'самолет'
        ]

        # Ключевые слова для высокого влияния
        self.high_impact_keywords = [
            # Рыночные термины и индикаторы
            r'акци[ияйюе]', r'котировк[иау]', r'бирж[ауе]', r'ммвб', r'индекс[ауы]?',
            r'фондов[ауы]{0,2}\s+рынок', r'цен[ауы]{0,2}\s+акци[ий]', r'обвал\s+котировок',
            r'падени[еяю]\s+акци[ий]', r'обвал\s+индекса', r'разворот\s+рынка',
            r'коррекци[яию]\s+рынка', r'тренд\s+на\s+рост', r'нисходящ[ийая]{0,2}\s+тренд',
            r'лонг[иау]?', r'шорт[ыау]?', r'лонгов[а-я]+\s+позици[яию]', r'шортов[а-я]+\s+позици[яию]',
            r'растущ[ийая]{0,2}\s+тренд', r'коррекци[яию]', r'ценн[ыеая]{0,2}\s+бумаг[иа]',
            r'облигаци[ияю]', r'евробонд[ыау]?', r'фьючерс[ыау]?', r'опцион[ыау]?', r'дериватив[ыау]?',
            r'торгов[а-я]+\s+сессия', r'биржев[а-я]+\s+сессия', r'инвестиц[а-я]+',
            r'инвестор[ауы]?', r'капитализаци[яию]', r'дивиденд[ауы]?', r'отчетност[ьию]',
            r'прибыл[ьию]', r'убыт[окки]', r'ebitda', r'free\s*float',
            
            # Макроэкономика
            r'ключев[аяуюо]+\s+ставк[аиу]', r'инфляци[яию]', r'ввп', 
            r'центральный\s+банк', r'курс[ауы]+\s+(доллар|евро|юан|валют)', 
            r'минфин', r'минэкономразвития', r'ставк[аиу]\s+по\s+ипотек[еи]',
            r'дефицит\s+бюджета', r'профицит\s+бюджета',
            
            # Геополитика и санкции
            r'санкци[ияю]', r'ограничени[еяю]', r'эмбарго', r'заморозк[аиу]\s+актив[а-я]*',
            r'арест[а-я]*\s+актив[а-я]*', r'недружественн[ыеая]{0,2}\s+страны',
            r'импортозамещени[еяю]', r'параллельн[ыеая]{0,2}\s+импорт',
            
            # Нефть, газ и энергетика
            r'нефт[ьи]', r'газ[ауе]?', r'brent', r'urals', r'опек', 
            r'добыч[аиу]\s+нефти', r'газопровод', r'северн[ыйаяое]+\s+поток',
            
            # Корпоративные события
            r'слияни[еяю]', r'поглощени[еяю]', r'ipo', r'листинг',
            r'размещени[еяю]', r'buy\s*back', r'отчетност[ьи]', 
            r'финансов[а-я]*\s+результат[ыау]?', r'капитализаци[яию]',
            
            # Регуляторы и законодательство
            r'правительств[оа]', r'госдум[аы]', r'законопроект', 
            r'федеральн[ыйаяое]+\s+закон', r'постановлени[еяю]', 
            r'лицензи[яию]', r'налог[а-я]*', r'пошлин[ауы]?', r'тариф[а-я]*',
        ]
        
        # Ключевые слова для среднего влияния
        self.medium_impact_keywords = [
            # Отраслевые новости
            r'отрасл[ьи]', r'сектор[ауе]?', r'промышленност[ьи]', 
            r'производств[оа]', r'машиностроени[еяю]', 
            r'легк[ауюо]{2,3}\s+промышленност[ьи]',
            r'тяжел[ауюо]{2,3}\s+промышленност[ьи]', 
            r'строительств[оа]', r'девелопмент', 
            r'недвижимост[ьи]', r'ретейл', r'розниц[ауе]',
            r'e-commerce', r'электронн[ауюо]{2,3}\s+торговл[яию]',
            
            # Компании и бизнес
            r'компани[яию]', r'предприяти[еяю]', r'бизнес[ауе]?',
            r'малый\s+бизнес', r'средний\s+бизнес', r'мсп',
            r'стартап[а-я]*',  # Добавили
            r'венчур[а-я]*',   # Добавили
            r'инвестиц',       # Тут пусть тоже будет (если хотим дублировать «инвестиции»)
            r'акционер', r'совет\s+директоров',
            
            # Цифровизация и технологии
            # (Убрали дубликат "импортозамещени[еяю]")
            r'технологи', r'инноваци',
            r'искусственн[ыоего]{2,3}\s+интеллект',
            r'нейросет[ьи]', r'блокчейн'
            
            # Рынок труда
            r'безработиц[ауе]', r'занятост[ьи]', r'рынок\s+труда',
            r'зарплат[ауе]', r'оплат[ауе]\s+труда', r'ваканси[яию]'
        ]
                
        # Компиляция регулярных выражений
        self.exclusion_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.exclusion_keywords]
        self.blue_chips_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.blue_chips]
        self.high_impact_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.high_impact_keywords]
        self.medium_impact_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.medium_impact_keywords]

    def should_exclude(self, text):
        """
        Проверяет, содержит ли текст слова из черного списка
        
        Returns:
        --------
        bool
            True если текст нужно исключить, False если можно оставить
        matched_words : list
            Список найденных слов из черного списка
        """
        excluded_matches = []
        for pattern in self.exclusion_patterns:
            matches = pattern.findall(text)
            if matches:
                # Если matches - список кортежей, берем первый элемент каждого кортежа
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                excluded_matches.extend(matches)
        
        return bool(excluded_matches), excluded_matches

    def classify_news(self, text):
        """
        Классифицирует новость по уровню потенциального влияния на рынок
        
        Returns:
        --------
        impact_level : str
            'HIGH', 'MEDIUM', 'LOW' или 'EXCLUDED'
        matched_keywords : list
            Список найденных ключевых слов
        excluded_words : list
            Список найденных слов из черного списка (если есть)
        """
        # Сначала проверяем на исключения
        should_exclude, excluded_words = self.should_exclude(text)
        if should_exclude:
            return 'EXCLUDED', [], excluded_words
        
        # Если текст не исключен, продолжаем обычную классификацию
        # Проверяем упоминания голубых фишек
        blue_chips_matches = []
        for pattern in self.blue_chips_patterns:
            matches = pattern.findall(text)
            if matches:
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                blue_chips_matches.extend(matches)
        
        # Проверяем на высокое влияние
        high_impact_matches = []
        for pattern in self.high_impact_patterns:
            matches = pattern.findall(text)
            if matches:
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                high_impact_matches.extend(matches)
        
        if blue_chips_matches or high_impact_matches:
            all_matches = blue_chips_matches + high_impact_matches
            return 'HIGH', all_matches, []
        
        # Проверяем на среднее влияние
        medium_impact_matches = []
        for pattern in self.medium_impact_patterns:
            matches = pattern.findall(text)
            if matches:
                if matches and isinstance(matches[0], tuple):
                    matches = [m[0] for m in matches]
                medium_impact_matches.extend(matches)
        
        if medium_impact_matches:
            return 'MEDIUM', medium_impact_matches, []
        
        return 'LOW', [], []

def process_news_file(filename):
    """Обрабатывает файл с новостями и классифицирует их"""
    # Читаем файл
    df = pd.read_csv(filename)
    
    # Создаем классификатор
    classifier = MarketImpactClassifier()
    
    # Классифицируем новости
    results = []
    for _, row in df.iterrows():
        impact_level, keywords, excluded_words = classifier.classify_news(row['text'])
        results.append({
            'date': row['date'],
            'time': row['time'],
            'text': row['text'],
            'impact_level': impact_level,
            'matched_keywords': ', '.join(set(keywords)) if keywords else None,
            'excluded_words': ', '.join(set(excluded_words)) if excluded_words else None
        })
    
    # Создаем новый датафрейм с результатами
    results_df = pd.DataFrame(results)
    
    # Выводим начальную статистику
    print("\nНачальная статистика по уровням влияния:")
    print(results_df['impact_level'].value_counts())
    
    # Удаляем новости с низким уровнем влияния и исключенные новости
    results_df = results_df[~results_df['impact_level'].isin(['LOW', 'EXCLUDED'])]
    
    # Выводим статистику после фильтрации
    print("\nСтатистика после фильтрации:")
    print(results_df['impact_level'].value_counts())
    
    # Выводим примеры для оставшихся категорий
    for level in ['HIGH', 'MEDIUM']:
        print(f"\nПримеры новостей с уровнем влияния {level}:")
        examples = results_df[results_df['impact_level'] == level].iloc[:2]
        for _, row in examples.iterrows():
            print(f"\n[{row['date']} {row['time']}]")
            if row['matched_keywords']:
                print(f"Ключевые слова: {row['matched_keywords']}")
            print(row['text'])
    
    # Сохраняем результаты
    output_filename = 'market_impact_news_test.csv'
    results_df.to_csv(output_filename, index=False)
    print(f"\nРезультаты сохранены в {output_filename}")
    
    return results_df

In [ ]:
if __name__ == "__main__":
    results_df = process_news_file('../chunk_news.csv')

In [ ]:
if __name__ == "__main__":
    results_df = process_news_file('../merged_df/news_tg.csv')

In [ ]:
hh = pd.read_csv('market_impact_news.csv')
hh.head()

In [ ]:
self.exclusion_keywords = [
    # 1. Криптовалюты
    r'\bкриптовалют',      # криптовалюта, криптовалюты, криптовалютном...
    r'\bкрипта\b',         # «крипта» в качестве самостоятельного слова
    r'\bбиткоин',          # биткоин
    r'\bbitcoin',          # Bitcoin
    r'\bэфир(иум)?',       # эфир, эфириум
    r'\beth(ereum)?',      # ETH, Ethereum
    r'\bnft\b',            # NFT
    r'\bдеф[аи]\b',        # DeFi (дефа, defi)
    r'\bстейблкоин',       # стейблкоин
    r'\bstablecoin',       
    r'\bмайнинг',          # майнинг
    r'\b(mining)\b',
    r'\bбинанс\b',         # Binance
    r'\bbinance\b',
    r'\bх[уа]оби\b',       # Huobi
    r'\bbybit\b',
    r'\bokx\b',
    # Если у вас в статьях часто встречаются специфические проекты,
    # можно перечислить и их.

    # 2. Спорт, соревнования
    r'\bчемпионат', r'\bтурнир', r'\bматч[аи]?', r'\bлига\b', r'\bфутбол',
    r'\bбаскетбол', r'\bхоккей', r'\bволейбол', 
    r'\bформул[аи]-?\d',   # формула-1, формула 1
    r'\bmotogp\b',         # MotoGP
    r'\bufc\b',            # UFC 
    r'\bбокс\b', r'\bпромоутер\b', 

    # 3. Развлечения, шоу-бизнес, светская хроника
    r'\bконцерт', r'\bфестивал', r'\bсериал', r'\bкино\b', r'\bфильм',
    r'\bакт[её]р', r'\bактрис', r'\bпев[иец|ица]', r'\bартист', r'\bшоубизнес',
    r'\bcelebrity', r'\bселебрити', r'\bзвезд[аыя]', r'\bскандал',
    r'\bмузык[а-я]*\b',
    r'\bевровидени[еяю]\b',

    # 4. Гемблинг, казино
    r'\bказино\b', r'\bбукмекер', r'\bставк[иауе]\s+на\s+спорт',
    r'\bлотере[яию]', r'\bрулетк[а-я]*',
    r'\bигров[а-я]*\s+автомат',

    # 5. Авто / мотоспорт (если точно не влияет)
    r'\bmotorsport', r'\bралли\s+рейд',  # (не путать с рыночным «ралли», но тут \bралли\s+рейд)
    r'\bавтоспорт', r'\bавтогонки', r'\bmotocross', 
    r'\bsuperbike', r'\bmotogp\b',  # дублируется, можно оставить
    r'\bdrift\b', r'\bдрифт',
    
    # 6. Светская хроника, gossip
    r'\bсплетни\b', r'\bслухи\b', r'\bтаблоид\b', r'\bжелтая\s+пресса',
    r'\bgossip\b',

    # 7. (Опционально) Исключать зарубежных автопроизводителей:
    #    (если точно уверены, что вам это не важно)
    r'\btesla\b', r'\bмерседес\b', r'\bmercedes\b', 
    r'\bbmw\b', r'\baudi\b', r'\btoyota\b', r'\bnissan\b', r'\bhonda\b',
    
    # 8. (Опционально) Другие нерелевантные темы
    r'\bмода\b', r'\bfashion\b',
    r'\bкосметик[а-я]*\b',
    r'\bпокер\b', r'\bbst\b', # betting
    r'\bастролог', r'\bгороскоп',
    # ... в зависимости от вашей специфики
]